![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.0.Clinical_Text_Classification_with_SparkNLP.ipynb)

# Colab Setup

In [ ]:
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

In [ ]:
from johnsnowlabs import nlp, medical
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="gpu")

In [5]:
spark

In [ ]:
!pip install -q git+https://github.com/tensorflow/addons.git

In [39]:
!pip list | grep tensorflow

tensorflow                               2.19.0
tensorflow-addons                        0.23.0.dev0
tensorflow-datasets                      4.9.9
tensorflow_decision_forests              1.12.0
tensorflow-hub                           0.16.1
tensorflow-metadata                      1.17.2
tensorflow-probability                   0.25.0
tensorflow-text                          2.19.0


## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).

Here are some Multi-label Text Classification models that trained with MultiClassifierDL:

|index|model|
|-----:|:-----|
|1|[multiclassifierdl_heart_disease_en](https://nlp.johnsnowlabs.com/2023/10/16/multiclassifierdl_heart_disease_en.html)|
|2|[multiclassifierdl_hoc_en](https://nlp.johnsnowlabs.com/2023/07/04/multiclassifierdl_hoc_en.html)|
|3|[multiclassifierdl_litcovid_en](https://nlp.johnsnowlabs.com/2023/07/04/multiclassifierdl_litcovid_en.html)|
|4|[multiclassifierdl_respiratory_disease_en](https://nlp.johnsnowlabs.com/2023/10/03/multiclassifierdl_respiratory_disease_en.html)|


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

Here are some Social Determinants of Health (SDOH) models that trained with GenericClassifier:

|index|model|
|-----:|:-----|
|1|[genericclassifier_sdoh_economics_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_economics_binary_sbiobert_cased_mli_en.html)|
|2|[genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli_en.html)|
|3|[genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli_en.html)|
|4|[genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli_en.html)|
|5|[genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli_en.html)|
|6|[genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli_en.html)
|7|[genericclassifier_sdoh_mental_health_clinical](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_mental_health_clinical_en.html)
|8|[genericclassifier_sdoh_under_treatment_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_under_treatment_sbiobert_cased_mli_en.html)
|9|[genericclassifier_patient_complaint_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/08/31/patient_complaint_classifier_generic_bert_M1_en.html)
|10|[Genericclassifier_age_group_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/08/16/genericclassifier_age_group_sbiobert_cased_mli_en.html)

## GenericLogRegClassifier

`GenericLogRegClassifier` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.

|index|model|
|-----:|:-----|
|1|[generic_logreg_classifier_ade](https://nlp.johnsnowlabs.com/2023/05/09/generic_logreg_classifier_ade_en.html)
|2|[generic_logreg_classifier_metastasis](https://nlp.johnsnowlabs.com/2024/08/09/generic_logreg_classifier_metastasis_en.html)



## GenericSVMClassifier

`GenericSVMClassifier` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

|index|model|
|-----:|:-----|
|1|[generic_svm_classifier_ade](https://nlp.johnsnowlabs.com/2023/05/09/generic_svm_classifier_ade_en.html)
|2|[generic_svm_classifier_metastasis](https://nlp.johnsnowlabs.com/2024/08/09/generic_svm_classifier_metastasis_en.html)

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier).

|index|model|
|-----:|:-----|
|1|[classifier_logreg_ade](https://nlp.johnsnowlabs.com/2023/05/16/classifier_logreg_ade_en.html)|


# ADE Dataset

### Data Preprocessing

In [6]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE Negative Dataset**

In [7]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg.head()

,col1
0,6460590 NEG Clioquinol intoxication occurring ...
1,"8600337 NEG ""Retinoic acid syndrome"" was preve..."
2,8402502 NEG BACKGROUND: External beam radiatio...
3,"8700794 NEG Although the enuresis ceased, she ..."
4,17662448 NEG A 42-year-old woman had uneventfu...


In [8]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE Positive Dataset**

In [9]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos.head()

,0,1,2,3,4,5,6,7
0,10030778,Intravenous azithromycin-induced ototoxicity.,ototoxicity,43,54,azithromycin,22,34
1,10048291,"Immobilization, while Paget's bone disease was...",increased calcium-release,960,985,dihydrotachysterol,908,926
2,10048291,Unaccountable severe hypercalcemia in a patien...,hypercalcemia,31,44,dihydrotachysterol,94,112
3,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,naproxen,646,654
4,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,oxaprozin,659,668


In [10]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging Positive and Negative dataset**

In [11]:
ade_df= pd.concat([df_neg, df_pos])
ade_df.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [12]:
ade_df["category"].value_counts()

,count
category,
neg,16695
pos,6821


In [13]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


We take 30% of the data to make a faster run. You can use all data for better scores.

In [14]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5606
Test Dataset Count: 1473


In [15]:
spark_df.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|     neg| 5028|
|     pos| 2051|
+--------+-----+



In [16]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



In [17]:
spark_df.head(3)

[Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg'),
 Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg'),
 Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')]

## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [18]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [19]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| 'Bail-out' bivalirudin use in patients with thrombotic c...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin us...|
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [20]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 15-year-old boy had temporary hypertropia, supraductio...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had tem...|
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [21]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [22]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[0.036828704, 0.052321058, 0.11382609, -0.05798419, -0.1450678, 0.13031518, 0.13105926, 0.077215254, -0.11359191, -0...|
|[[0.04241796, 0.10478715, 0.042885326, 0.06311753, -0.3185922, 0.046458624, 0.17697075, -0.037831675, -0.01754189, -0...|
|[[0.057804022, 0.047698125, 0.06304918, 0.15662706, -0.04416214, -0.06267688, 0.08489624, -0.0543084, -0.0027059284, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [23]:
log_folder="ADE_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDL

In [24]:
classifier_dl = nlp.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setLabelColumn("category")\
    .setBatchSize(16)\
    .setMaxEpochs(30)\
    .setLr(0.002)\
    .setDropout(0.3)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
    # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [25]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [26]:
!cat $log_folder/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5606 - classes: 2
Epoch 0/30 - 2.03s - loss: 187.2653 - acc: 0.7508929 - batches: 351
Epoch 1/30 - 1.57s - loss: 171.21953 - acc: 0.8057143 - batches: 351
Epoch 2/30 - 1.57s - loss: 166.90343 - acc: 0.82910717 - batches: 351
Epoch 3/30 - 1.53s - loss: 165.06546 - acc: 0.84375 - batches: 351
Epoch 4/30 - 1.56s - loss: 163.14418 - acc: 0.8519643 - batches: 351
Epoch 5/30 - 1.57s - loss: 161.43214 - acc: 0.85732144 - batches: 351
Epoch 6/30 - 1.62s - loss: 159.96835 - acc: 0.8619643 - batches: 351
Epoch 7/30 - 1.46s - loss: 158.72842 - acc: 0.86642855 - batches: 351
Epoch 8/30 - 1.42s - loss: 157.59912 - acc: 0.8692857 - batches: 351
Epoch 9/30 - 1.45s - loss: 156.61308 - acc: 0.87285715 - batches: 351
Epoch 10/30 - 1.42s - loss: 155.77753 - acc: 0.87464285 - batches: 351
Epoch 11/30 - 1.39s - loss: 155.01375 - acc: 0.87642854 - batches: 351
Epoch 12/30 - 1.38s - loss: 154.33766 - acc: 0.87892854 - 

In [27]:
preds = clfDL_model_hc100.transform(testData_with_embeddings).cache()

In [28]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [29]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.88      0.89      0.88      1057
         pos       0.70      0.68      0.69       416

    accuracy                           0.83      1473
   macro avg       0.79      0.78      0.79      1473
weighted avg       0.83      0.83      0.83      1473



In [30]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["pos-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]
results_df

,pos-f1-score,accuracy
ClassifierDL_100d,0.690244,0.827563


### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier.

In [31]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [32]:
multiClassifier = nlp.MultiClassifierDLApproach()\
    .setInputCols("sentence_embeddings")\
    .setOutputCol("prediction")\
    .setLabelColumn("category_array")\
    .setBatchSize(16)\
    .setMaxEpochs(20)\
    .setLr(9e-3)\
    .setThreshold(0.5)\
    .setShufflePerEpoch(False)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
  #   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [33]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [34]:
!cat $log_folder/MultiClassifierDLApproach_*

Training started - epochs: 20 - learning_rate: 0.009 - batch_size: 16 - training_examples: 5606 - classes: 2
Epoch 0/20 - 7.39s - loss: 0.5211027 - acc: 0.7369643 - batches: 351
Epoch 1/20 - 2.84s - loss: 0.45771593 - acc: 0.7746429 - batches: 351
Epoch 2/20 - 2.74s - loss: 0.42100212 - acc: 0.79821426 - batches: 351
Epoch 3/20 - 2.78s - loss: 0.3972907 - acc: 0.8149107 - batches: 351
Epoch 4/20 - 2.75s - loss: 0.37783632 - acc: 0.8275 - batches: 351
Epoch 5/20 - 2.76s - loss: 0.36196303 - acc: 0.8369643 - batches: 351
Epoch 6/20 - 2.70s - loss: 0.34776407 - acc: 0.84232146 - batches: 351
Epoch 7/20 - 2.75s - loss: 0.33263168 - acc: 0.8544643 - batches: 351
Epoch 8/20 - 2.71s - loss: 0.3181401 - acc: 0.8584821 - batches: 351
Epoch 9/20 - 2.77s - loss: 0.3043693 - acc: 0.8619643 - batches: 351
Epoch 10/20 - 2.89s - loss: 0.29091737 - acc: 0.8699107 - batches: 351
Epoch 11/20 - 2.81s - loss: 0.2727908 - acc: 0.88035715 - batches: 351
Epoch 12/20 - 2.81s - loss: 0.26008934 - acc: 0.88875 

In [35]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings).cache()

In [36]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [37]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

,count
result,
['neg'],962
['pos'],508
"['neg', 'pos']",3


In [38]:
preds_df[preds_df.result.apply(len)==2]

,category,text,result,metadata
4,neg,A novel approach to central venous catheter t...,"[neg, pos]","[{'sentence': '0', 'neg': '0.500581', 'pos': '..."
219,neg,Physicians should remain alert to the potenti...,"[neg, pos]","[{'sentence': '0', 'neg': '0.50044626', 'pos':..."
1414,pos,Severe osteomalacia was present in two epilept...,"[neg, pos]","[{'sentence': '0', 'neg': '0.5003451', 'pos': ..."


In [39]:
preds_df[preds_df.result.apply(len)==0]

,category,text,result,metadata


MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will not keep zero label predictions and get the highest score as prediction.

In [40]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

,count
result,
neg,963
pos,510


In [41]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.90      0.82      0.86      1057
         pos       0.63      0.77      0.69       416

    accuracy                           0.81      1473
   macro avg       0.76      0.79      0.77      1473
weighted avg       0.82      0.81      0.81      1473



### Generic Classifier

In [49]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [50]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [51]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [52]:
!cat $log_folder/GenericClassifierApproach_*

Training 25 epochs
Epoch 1/25	0.37s	Loss: 12.376424	ACC: 0.62905943
Epoch 2/25	0.14s	Loss: 10.290136	ACC: 0.70967025
Epoch 3/25	0.14s	Loss: 9.664499	ACC: 0.7247208
Epoch 4/25	0.15s	Loss: 9.510019	ACC: 0.73621327
Epoch 5/25	0.15s	Loss: 9.456532	ACC: 0.72867227
Epoch 6/25	0.14s	Loss: 9.180353	ACC: 0.7478519
Epoch 7/25	0.14s	Loss: 8.915824	ACC: 0.75503427
Epoch 8/25	0.14s	Loss: 8.807046	ACC: 0.7607265
Epoch 9/25	0.15s	Loss: 8.642652	ACC: 0.764765
Epoch 10/25	0.14s	Loss: 8.556572	ACC: 0.77279687
Epoch 11/25	0.13s	Loss: 8.430639	ACC: 0.77456206
Epoch 12/25	0.15s	Loss: 8.353577	ACC: 0.77506334
Epoch 13/25	0.15s	Loss: 8.245626	ACC: 0.78642696
Epoch 14/25	0.14s	Loss: 8.309855	ACC: 0.7886029
Epoch 15/25	0.13s	Loss: 8.194262	ACC: 0.7851841
Epoch 16/25	0.13s	Loss: 8.145708	ACC: 0.7805572
Epoch 17/25	0.15s	Loss: 8.386643	ACC: 0.78012544
Epoch 18/25	0.15s	Loss: 7.9686747	ACC: 0.7881086
Epoch 19/25	0.13s	Loss: 7.9342237	ACC: 0.7959698
Epoch 20/25	0.15s	Loss: 7.929268	ACC: 0.7854939
Epoch 21/25	0.15s

In [53]:
pred_df = generic_model_hc100.transform(testData_with_embeddings).cache()

In [54]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [55]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.80      0.84      1057
         pos       0.59      0.73      0.65       416

    accuracy                           0.78      1473
   macro avg       0.74      0.77      0.75      1473
weighted avg       0.80      0.78      0.79      1473



In [56]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [57]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [58]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [59]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [60]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 20 epochs
Epoch 1/20	0.15s	Loss: 26.451231	ACC: 0.69675386
Epoch 2/20	0.05s	Loss: 24.540642	ACC: 0.7164592
Epoch 3/20	0.05s	Loss: 23.557869	ACC: 0.7252012
Epoch 4/20	0.05s	Loss: 23.022427	ACC: 0.7330625
Epoch 5/20	0.05s	Loss: 22.531796	ACC: 0.738525
Epoch 6/20	0.05s	Loss: 22.372961	ACC: 0.74149126
Epoch 7/20	0.05s	Loss: 22.162153	ACC: 0.7470442
Epoch 8/20	0.05s	Loss: 22.018856	ACC: 0.7487711
Epoch 9/20	0.05s	Loss: 21.850657	ACC: 0.75188696
Epoch 10/20	0.05s	Loss: 21.780178	ACC: 0.7602287
Epoch 11/20	0.06s	Loss: 21.719423	ACC: 0.7551317
Epoch 12/20	0.05s	Loss: 21.566282	ACC: 0.7595637
Epoch 13/20	0.06s	Loss: 21.574396	ACC: 0.75934434
Epoch 14/20	0.06s	Loss: 21.431873	ACC: 0.7584148
Epoch 15/20	0.05s	Loss: 21.375307	ACC: 0.7601347
Epoch 16/20	0.05s	Loss: 21.538742	ACC: 0.7573042
Epoch 17/20	0.06s	Loss: 21.426434	ACC: 0.75676805
Epoch 18/20	0.06s	Loss: 21.402674	ACC: 0.7580597
Epoch 19/20	0.05s	Loss: 21.239674	ACC: 0.76085186
Epoch 20/20	0.05s	Loss: 21.400162	ACC: 0.7591668
Train

In [61]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [62]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [63]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.80      0.92      0.86      1057
         pos       0.68      0.41      0.51       416

    accuracy                           0.78      1473
   macro avg       0.74      0.67      0.68      1473
weighted avg       0.76      0.78      0.76      1473



In [64]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [65]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [66]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.015)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [67]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [68]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.14s	Loss: 26.475868	ACC: 0.7141544
Epoch 2/25	0.05s	Loss: 24.954428	ACC: 0.7278681
Epoch 3/25	0.05s	Loss: 24.503813	ACC: 0.73959374
Epoch 4/25	0.05s	Loss: 24.271524	ACC: 0.74509805
Epoch 5/25	0.05s	Loss: 23.95904	ACC: 0.74913657
Epoch 6/25	0.05s	Loss: 24.0294	ACC: 0.7482975
Epoch 7/25	0.06s	Loss: 24.122849	ACC: 0.7520576
Epoch 8/25	0.05s	Loss: 23.698452	ACC: 0.7629929
Epoch 9/25	0.04s	Loss: 23.828619	ACC: 0.7578334
Epoch 10/25	0.05s	Loss: 23.829199	ACC: 0.75521874
Epoch 11/25	0.05s	Loss: 23.921917	ACC: 0.752726
Epoch 12/25	0.05s	Loss: 23.790323	ACC: 0.75823724
Epoch 13/25	0.05s	Loss: 23.87044	ACC: 0.7531229
Epoch 14/25	0.05s	Loss: 23.619593	ACC: 0.759609
Epoch 15/25	0.06s	Loss: 23.594837	ACC: 0.75996757
Epoch 16/25	0.06s	Loss: 23.615671	ACC: 0.7605002
Epoch 17/25	0.05s	Loss: 23.816427	ACC: 0.75611
Epoch 18/25	0.05s	Loss: 23.620049	ACC: 0.76071954
Epoch 19/25	0.05s	Loss: 23.635164	ACC: 0.7634247
Epoch 20/25	0.05s	Loss: 23.577478	ACC: 0.75934786
Epoch 21/2

In [69]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [70]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [71]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.79      0.91      0.85      1057
         pos       0.65      0.39      0.49       416

    accuracy                           0.77      1473
   macro avg       0.72      0.65      0.67      1473
weighted avg       0.75      0.77      0.75      1473



In [72]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [73]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [74]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| 'Bail-out' bivalirudin use in patients with thrombotic c...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin us...|
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [75]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 15-year-old boy had temporary hypertropia, supraductio...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had tem...|
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [76]:
log_folder="ADE_logs_healthcare_200d"

### ClassifierDL

In [77]:
classifier_dl = nlp.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setBatchSize(16)\
    .setMaxEpochs(30)\
    .setLr(0.001)\
    .setDropout(0.3)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [78]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [79]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.88      0.90      0.89      1057
         pos       0.73      0.69      0.71       416

    accuracy                           0.84      1473
   macro avg       0.81      0.79      0.80      1473
weighted avg       0.84      0.84      0.84      1473



In [80]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [81]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [82]:
multiClassifier =nlp.MultiClassifierDLApproach()\
    .setInputCols("sentence_embeddings")\
    .setOutputCol("prediction")\
    .setLabelColumn("category_array")\
    .setBatchSize(32)\
    .setMaxEpochs(20)\
    .setLr(9e-3)\
    .setThreshold(0.5)\
    .setShufflePerEpoch(False)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
  #   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [83]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [84]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [85]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

,count
result,
['neg'],1069
['pos'],404


In [86]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

,count
result,
neg,1069
pos,404


In [87]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.89      0.90      0.89      1057
         pos       0.73      0.71      0.72       416

    accuracy                           0.84      1473
   macro avg       0.81      0.80      0.81      1473
weighted avg       0.84      0.84      0.84      1473



In [88]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [89]:
#from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [90]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [91]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [92]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.94      0.68      0.79      1057
         pos       0.53      0.89      0.66       416

    accuracy                           0.74      1473
   macro avg       0.73      0.79      0.73      1473
weighted avg       0.82      0.74      0.75      1473



In [93]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [94]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [95]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [96]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [97]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [98]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.83      0.92      0.87      1057
         pos       0.72      0.51      0.59       416

    accuracy                           0.80      1473
   macro avg       0.77      0.71      0.73      1473
weighted avg       0.79      0.80      0.79      1473



In [99]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [100]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [101]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [102]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [103]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [104]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.84      0.89      0.86      1057
         pos       0.66      0.58      0.62       416

    accuracy                           0.80      1473
   macro avg       0.75      0.73      0.74      1473
weighted avg       0.79      0.80      0.79      1473



In [105]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [106]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        bert_sent
])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [107]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| 'Bail-out' bivalirudin use in patients with thrombotic c...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin us...|
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [108]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 15-year-old boy had temporary hypertropia, supraductio...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had tem...|
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [109]:
log_folder="ADE_logs_bert"

### ClassifierDL

In [110]:
classifier_dl = nlp.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setBatchSize(32)\
    .setMaxEpochs(20)\
    .setLr(0.001)\
    .setDropout(0.3)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [111]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [112]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.90      0.94      0.92      1057
         pos       0.82      0.73      0.77       416

    accuracy                           0.88      1473
   macro avg       0.86      0.83      0.84      1473
weighted avg       0.88      0.88      0.88      1473



In [113]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [114]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [115]:
multiClassifier =nlp.MultiClassifierDLApproach()\
    .setInputCols("sentence_embeddings")\
    .setOutputCol("prediction")\
    .setLabelColumn("category_array")\
    .setBatchSize(32)\
    .setMaxEpochs(20)\
    .setLr(9e-3)\
    .setThreshold(0.5)\
    .setShufflePerEpoch(False)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
  #   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [116]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [117]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [118]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

,count
result,
['neg'],995
['pos'],477
"['neg', 'pos']",1


In [119]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

,count
result,
neg,996
pos,477


In [120]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.93      0.88      0.91      1057
         pos       0.73      0.84      0.78       416

    accuracy                           0.87      1473
   macro avg       0.83      0.86      0.84      1473
weighted avg       0.88      0.87      0.87      1473



In [121]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [123]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [124]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [125]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [126]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.96      0.86      0.91      1057
         pos       0.72      0.90      0.80       416

    accuracy                           0.87      1473
   macro avg       0.84      0.88      0.86      1473
weighted avg       0.89      0.87      0.88      1473



In [127]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [128]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [129]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [130]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [131]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [132]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.92      0.92      0.92      1057
         pos       0.80      0.80      0.80       416

    accuracy                           0.89      1473
   macro avg       0.86      0.86      0.86      1473
weighted avg       0.89      0.89      0.89      1473



In [133]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [134]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [135]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [136]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [137]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [138]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.89      0.95      0.92      1057
         pos       0.86      0.69      0.76       416

    accuracy                           0.88      1473
   macro avg       0.87      0.82      0.84      1473
weighted avg       0.88      0.88      0.87      1473



In [139]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [140]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = nlp.Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = nlp.StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

stemmer = nlp.Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(3)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(
    stages=[
        document_assembler,
        tokenizer,
        normalizer,
        stopwords_cleaner,
        stemmer,
        logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [141]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.89      0.88      1057
         pos       0.70      0.65      0.67       416

    accuracy                           0.82      1473
   macro avg       0.78      0.77      0.78      1473
weighted avg       0.82      0.82      0.82      1473



In [142]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["pos"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in ADE Dataset

Here is the all results of classifier models. Only positive ADE f1 scores and accuracy scores are presented.

In [143]:
results_df

,pos-f1-score,accuracy
ClassifierDL_100d,0.690244,0.827563
MultiClassifierDL_100d,0.691145,0.805838
GenericClassifier_100d,0.652361,0.780041
GenericLogReg_100d,0.511211,0.778004
GenericSVM_100d,0.489552,0.767821
ClassifierDL_200d,0.708075,0.840462
MultiClassifierDL_200d,0.719512,0.843856
GenericClassifier_200d,0.661319,0.742023
GenericLogReg_200d,0.593530,0.803802
GenericSVM_200d,0.617761,0.798371


# Mtsamples Dataset

**❗Please restart runtime and continue**

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="gpu")

## Load Dataset

In [7]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [8]:
spark_df = spark.read.csv("mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [9]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)



In [10]:
spark_df.count()

638

In [11]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|       Neurology|  143|
|      Orthopedic|  223|
|Gastroenterology|  157|
+----------------+-----+



In [12]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [13]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   95|
|       Neurology|  121|
|      Orthopedic|  188|
|Gastroenterology|  132|
+----------------+-----+



In [14]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   20|
|       Neurology|   22|
|      Orthopedic|   35|
|Gastroenterology|   25|
+----------------+-----+



## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [40]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [41]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [42]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [43]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [44]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.010433162, 0.0127568655, 0.110687375, 0.1609855, -0.14818177, -0.050856464, 0.064748034, -0.019294674, -0.002178...|
|[[-0.0055608996, 0.018396137, 0.12305377, 0.13861515, -0.18651922, -0.07559964, 0.12978752, -0.0046452526, -0.0031408...|
|[[-0.019461514, 0.060639214, 0.11142021, 0.12826534, -0.16216077, -0.10416892, 0.17136306, -0.020363769, 0.01169337, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [45]:
log_folder="Mt_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDL

In [46]:
classifier_dl = nlp.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setLabelColumn("category")\
    .setBatchSize(8)\
    .setMaxEpochs(50)\
    .setLr(0.005)\
    .setDropout(0.3)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
    # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [47]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [48]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [49]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.00      0.00      0.00        25
       Neurology       0.00      0.00      0.00        22
      Orthopedic       0.59      1.00      0.74        35
         Urology       0.42      0.90      0.57        20

        accuracy                           0.52       102
       macro avg       0.25      0.47      0.33       102
    weighted avg       0.29      0.52      0.37       102



In [50]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_100d,0.329027,0.367573,0.519608


### MultiClassifierDL

In [51]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [52]:
multiClassifier =nlp.MultiClassifierDLApproach()\
    .setInputCols("sentence_embeddings")\
    .setOutputCol("prediction")\
    .setLabelColumn("category_array")\
    .setBatchSize(8)\
    .setMaxEpochs(40)\
    .setLr(5e-3)\
    .setThreshold(0.5)\
    .setShufflePerEpoch(False)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
  #   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [53]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [54]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [55]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

,count
result,
['Orthopedic'],35
['Urology'],21
['Gastroenterology'],19
['Neurology'],17
[],5
"['Neurology', 'Orthopedic']",4
"['Orthopedic', 'Gastroenterology']",1


In [56]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

,count
result,
Orthopedic,36
Urology,21
Gastroenterology,20
Neurology,20


In [57]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.86      0.90        22
       Neurology       0.80      0.76      0.78        21
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      1.00      0.95        19

        accuracy                           0.89        97
       macro avg       0.89      0.88      0.88        97
    weighted avg       0.89      0.89      0.89        97



In [58]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [59]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

In [60]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [61]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [62]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [63]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.92      0.90        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.94      0.91      0.93        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.89       102
       macro avg       0.89      0.89      0.89       102
    weighted avg       0.89      0.89      0.89       102



In [64]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [65]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [66]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [67]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [68]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [69]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.88        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.84      0.91      0.88        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.85       102
       macro avg       0.85      0.84      0.85       102
    weighted avg       0.85      0.85      0.85       102



In [70]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [71]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [72]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [73]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [74]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [75]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.84      0.91        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.94      0.91      0.93        35
         Urology       0.76      0.95      0.84        20

        accuracy                           0.88       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.89      0.88      0.88       102



In [76]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [77]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [78]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [79]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [80]:
log_folder="Mt_logs_healthcare_200d"

### ClassifierDL

In [81]:
classifier_dl = nlp.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setBatchSize(8)\
    .setMaxEpochs(50)\
    .setLr(0.005)\
    .setDropout(0.1)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [82]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [83]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.00      0.00      0.00        25
       Neurology       0.00      0.00      0.00        22
      Orthopedic       0.34      1.00      0.51        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.34       102
       macro avg       0.09      0.25      0.13       102
    weighted avg       0.12      0.34      0.18       102



In [84]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL



In [85]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [86]:
multiClassifier =nlp.MultiClassifierDLApproach()\
    .setInputCols("sentence_embeddings")\
    .setOutputCol("prediction")\
    .setLabelColumn("category_array")\
    .setBatchSize(16)\
    .setMaxEpochs(20)\
    .setLr(9e-3)\
    .setThreshold(0.2)\
    .setShufflePerEpoch(False)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
  #   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [87]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [88]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [89]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

,count
result,
['Orthopedic'],32
['Gastroenterology'],19
['Urology'],18
['Neurology'],13
"['Neurology', 'Orthopedic']",11
[],4
"['Orthopedic', 'Gastroenterology']",3
"['Urology', 'Orthopedic']",1
"['Urology', 'Gastroenterology']",1


In [90]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

,count
result,
Orthopedic,38
Gastroenterology,22
Urology,20
Neurology,18


In [91]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.92      0.96        24
       Neurology       0.89      0.80      0.84        20
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.90      0.95      0.92        19

        accuracy                           0.91        98
       macro avg       0.91      0.90      0.91        98
    weighted avg       0.91      0.91      0.91        98



In [92]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [93]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [94]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(
    stages=[
        features_asm,
        gc_graph_builder,
        gen_clf
])


In [95]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [96]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.81      0.88      0.85        25
       Neurology       0.88      0.68      0.77        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.80      0.80      0.80        20

        accuracy                           0.84       102
       macro avg       0.84      0.83      0.83       102
    weighted avg       0.84      0.84      0.84       102



In [97]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [98]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [99]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(
    stages=[
        features_asm,
        gc_logreg_graph_builder,
        gen_clf
])


In [100]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [101]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [102]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.81      0.84      0.82        25
       Neurology       0.88      0.68      0.77        22
      Orthopedic       0.85      0.94      0.89        35
         Urology       0.80      0.80      0.80        20

        accuracy                           0.83       102
       macro avg       0.83      0.82      0.82       102
    weighted avg       0.84      0.83      0.83       102



In [103]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [104]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [105]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(
    stages=[
        features_asm,
        gc_svm_graph_builder,
        gen_clf
])


In [106]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [107]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [108]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.81      0.88      0.85        25
       Neurology       0.89      0.77      0.83        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.89      0.80      0.84        20

        accuracy                           0.86       102
       macro avg       0.87      0.85      0.86       102
    weighted avg       0.86      0.86      0.86       102



In [109]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [110]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
              bert_sent
])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [111]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [112]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [113]:
log_folder="Mt_logs_bert"

### ClassifierDL

In [114]:
classifier_dl = nlp.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setBatchSize(8)\
    .setMaxEpochs(30)\
    .setLr(0.002)\
    .setDropout(0.1)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [115]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [116]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.83      0.86      0.84        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.88      0.88      0.88       102



In [117]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [118]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [119]:
multiClassifier =nlp.MultiClassifierDLApproach()\
    .setInputCols("sentence_embeddings")\
    .setOutputCol("prediction")\
    .setLabelColumn("category_array")\
    .setBatchSize(8)\
    .setMaxEpochs(20)\
    .setLr(9e-3)\
    .setThreshold(0.5)\
    .setShufflePerEpoch(False)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
  #   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
])

In [120]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [121]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [122]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

,count
result,
['Orthopedic'],31
['Neurology'],24
['Gastroenterology'],22
['Urology'],20
[],5


In [123]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

,count
result,
Orthopedic,31
Neurology,24
Gastroenterology,22
Urology,20


In [124]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.92      0.96        24
       Neurology       0.75      0.86      0.80        21
      Orthopedic       0.94      0.85      0.89        34
         Urology       0.85      0.94      0.89        18

        accuracy                           0.89        97
       macro avg       0.88      0.89      0.89        97
    weighted avg       0.90      0.89      0.89        97



In [125]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [126]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [127]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(16)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(
    stages=[
        features_asm,
        gc_graph_builder,
        gen_clf
])


In [128]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [129]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.70      0.95      0.81        22
      Orthopedic       0.96      0.74      0.84        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.87       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.89      0.87      0.87       102



In [130]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [131]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [132]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(
    stages=[
        features_asm,
        gc_logreg_graph_builder,
        gen_clf
])


In [133]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [134]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [135]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.86      0.95      0.90        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.88      0.88      0.88       102



In [136]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCLogReg_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [137]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [138]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(
    stages=[
        features_asm,
        gc_svm_graph_builder,
        gen_clf
])


In [139]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [140]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [141]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.92      0.92        25
       Neurology       0.75      0.95      0.84        22
      Orthopedic       0.91      0.86      0.88        35
         Urology       0.88      0.70      0.78        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.87      0.86      0.86       102



In [142]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCSVM_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [143]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [144]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.91      0.89      0.90        35
         Urology       0.79      0.95      0.86        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.87      0.86      0.86       102



In [145]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in Mtsamples Dataset

Here is the all results of classifier models. Macro f1, weighted f1 and accuracy scores are presented.

In [146]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_100d,0.329027,0.367573,0.519608
MultiClassifierDL_100d,0.884165,0.885508,0.886598
GenericClassifier_100d,0.886920,0.892282,0.892157
GenericLogReg_100d,0.848103,0.852503,0.852941
GenericSVM_100d,0.875801,0.884105,0.882353
ClassifierDL_200d,0.127737,0.175326,0.343137
MultiClassifierDL_200d,0.906453,0.907969,0.908163
GenericClassifier_200d,0.829874,0.840400,0.843137
GenericLogReg_200d,0.821163,0.830662,0.833333
GenericSVM_200d,0.855409,0.861605,0.862745
